In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import seaborn as sns
from datetime import datetime, timedelta
from scipy.stats import pearsonr
from scipy.stats import kendalltau
from scipy.stats import spearmanr
import os

In [2]:
path = x
filename = 'privatkunden.iid.Thalia.ohneAppleEtc.csv'
df_relevante_iids = pd.read_csv(os.path.join(path, filename))

In [3]:
path = x
filename = 'Thalia_transactions_zweiteBereinigung.csv'
df_transactions = pd.read_csv(os.path.join(path, filename))

In [4]:
path = x
filename = 'Thalia_history_ersteBereinigung.csv'
df_history = pd.read_csv(os.path.join(path, filename))

#### 1. Weitere Verarbeitung Transaktionen

In [16]:
df_t_merged = pd.merge(df_transactions, df_relevante_iids, on =["iid"])

In [17]:
df_t_merged_grouped = df_t_merged.copy()
df_t_merged_grouped["month_count"] = df_t_merged_grouped["month_count"] -16
df_t_merged_grouped = df_t_merged_grouped.groupby(["iid", "month_count"]).agg(count=("iid", "count"))
df_t_merged_grouped = df_t_merged_grouped.reset_index()

In [19]:
df_t_merged_grouped.head()

,iid,month_count,count
0,100056041,1,1
1,100056041,14,1
2,100057957,1,1
3,100057957,2,1
4,100057957,4,1


In [20]:
df_pivoted_t = df_t_merged_grouped.pivot(index="iid", columns="month_count", values="count")
df_pivoted_t = df_pivoted_t.reset_index()
df_pivoted_t = df_pivoted_t.fillna(0)

In [21]:
df_pivoted_t.head()

month_count,iid,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,100056041,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,100057957,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
2,100059553,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100062565,4.0,0.0,1.0,3.0,2.0,1.0,1.0,2.0,1.0,1.0,0.0,1.0,2.0,3.0,0.0
4,100062713,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### 1.1 LY berechnen

In [38]:
df_LY = df_pivoted_t.copy()
df_LY = df_LY.rename(columns={col: str(col) for col in df_LY.columns})
df_LY[df_LY.columns.difference(['iid'])] = df_LY[df_LY.columns.difference(['iid'])].applymap(lambda x: 1.0 if x != 0.0 else x)

In [39]:
df_LY.head()

month_count,iid,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,100056041,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,100057957,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
2,100059553,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100062565,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
4,100062713,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
# neue Spaltennamen für df_LY_anwendung definieren
new_columns = ["iid"]

# leeres DataFrame mit den neuen Spaltennamen erstellen
df_LY_anwendung = pd.DataFrame(columns=new_columns)

# Spalte "iid" aus df_LY kopieren
df_LY["0"] = 1.0
df_LY_anwendung["0"] = 1.0
df_LY_anwendung["iid"] = df_LY["iid"]

# Schleife über alle Spalten außer "iid" in df_LY
for i in range(1, 16):
    # die vorherige Spalte aus df_LY für den aktuellen Monat
    previous_month = i-1
    previous_month = str(previous_month)
    
    # neue Spalte für den aktuellen Monat in df_LY_anwendung
    current_month = i
    
    # Schleife über alle Zeilen in df_LY
    for index, row in df_LY.iterrows():
        # Prüfen, ob die vorherige Spalte in df_LY ungleich 0 ist
        if row[previous_month] != 0.0:
            df_LY_anwendung.at[index, str(current_month)] = 1.0
        else:
            # Wenn die vorherige Spalte in df_LY gleich 0 ist, um 1 erhöhen
            df_LY_anwendung.at[index, str(current_month)] = df_LY_anwendung.at[index, str(current_month-1)] + 1
            
df_LY_anwendung = df_LY_anwendung.drop("0", axis=1)

In [41]:
df_LY_anwendung.head()

,iid,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,100056041,1.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,1.0
1,100057957,1.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,3.0,4.0,1.0,2.0,1.0
2,100059553,1.0,1.0,2.0,3.0,4.0,5.0,1.0,2.0,3.0,1.0,2.0,3.0,4.0,5.0,6.0
3,100062565,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0
4,100062713,1.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0


#### 2. Weitere Verarbeitung History

In [43]:
pd.merge(df_history, df_relevante_iids, on =["iid"])["iid"].nunique()

713

In [28]:
df_h_merged = pd.merge(df_history, df_relevante_iids, on =["iid"])

In [23]:
df_h_merged_grouped = df_h_merged.copy()
df_h_merged_grouped = df_h_merged_grouped[df_h_merged_grouped["month_count"] >= 17]
df_h_merged_grouped["month_count"] = df_h_merged_grouped["month_count"] -16

C:\Users\felix.oechslein\AppData\Local\Temp\ipykernel_14240\1048610325.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_h_merged_grouped["month_count"] = df_h_merged_grouped["month_count"] -16


In [24]:
df_h_merged_grouped = df_h_merged_grouped.groupby(["iid", "month_count", "attrtype"]).agg(count=("iid", "count"))
df_h_merged_grouped = df_h_merged_grouped.reset_index()

##### 2.1 Pivot table für HY

In [25]:
df_h_merged_grouped_HY = df_h_merged_grouped[df_h_merged_grouped["attrtype"] == "HY"]
df_h_merged_grouped_HY = df_h_merged_grouped_HY[["iid", "month_count", "count"]]

In [26]:
df_pivoted_HY = df_h_merged_grouped_HY.pivot(index="iid", columns="month_count", values="count")
df_pivoted_HY = df_pivoted_HY.reset_index()
df_pivoted_HY = df_pivoted_HY.fillna(0)

In [27]:
df_pivoted_HY.head()

month_count,iid,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,100056041,2.0,2.0,2.0,3.0,2.0,3.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,3.0,1.0
1,100057957,2.0,2.0,2.0,3.0,2.0,3.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,3.0,1.0
2,100059553,2.0,2.0,2.0,3.0,2.0,3.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,3.0,1.0
3,100062565,2.0,2.0,2.0,3.0,2.0,3.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,3.0,1.0
4,100062713,2.0,2.0,2.0,3.0,2.0,3.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,3.0,1.0


##### 2.2 Pivot table für NO

In [28]:
df_h_merged_grouped_NO = df_h_merged_grouped[df_h_merged_grouped["attrtype"] == "NO"]
df_h_merged_grouped_NO = df_h_merged_grouped_NO[["iid", "month_count", "count"]]

In [29]:
df_pivoted_NO = df_h_merged_grouped_NO.pivot(index="iid", columns="month_count", values="count")
df_pivoted_NO = df_pivoted_NO.reset_index()
df_pivoted_NO = df_pivoted_NO.fillna(0)

In [30]:
df_pivoted_NO.head()

month_count,iid,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,100056041,1.0,1.0,1.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1,100057957,2.0,2.0,2.0,3.0,2.0,3.0,2.0,0.0,2.0,1.0,1.0,2.0,2.0,3.0,0.0
2,100059553,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
3,100062565,2.0,2.0,2.0,3.0,2.0,3.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,3.0,1.0
4,100062713,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### 2.3 Pivot table für LO

In [32]:
df_LO = df_pivoted_NO
df_LO = df_LO.rename(columns={col: str(col) for col in df_LO.columns})
df_LO[df_LO.columns.difference(['iid'])] = df_LO[df_LO.columns.difference(['iid'])].applymap(lambda x: 1.0 if x != 0.0 else x)

In [33]:
df_LO.head()

month_count,iid,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,100056041,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1,100057957,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
2,100059553,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
3,100062565,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,100062713,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
# neue Spaltennamen für df_LY_anwendung definieren
new_columns = ["iid"]

# leeres DataFrame mit den neuen Spaltennamen erstellen
df_LO_anwendung = pd.DataFrame(columns=new_columns)

# Spalte "iid" aus df_LY kopieren
df_LO["0"] = 1.0
df_LO_anwendung["0"] = 0.0
df_LO_anwendung["iid"] = df_LO["iid"]

# Schleife über alle Spalten außer "iid" in df_LY
for i in range(1, 16):
    # die vorherige Spalte aus df_LY für den aktuellen Monat
    previous_month = i-1
    
    # neue Spalte für den aktuellen Monat in df_LY_anwendung
    current_month = i
    
    # Schleife über alle Zeilen in df_LY
    for index, row in df_LO.iterrows():
        # Prüfen, ob die vorherige Spalte in df_LY ungleich 0 ist
        if row[str(previous_month)] != 0.0:
            df_LO_anwendung.at[index, str(current_month)] = 1.0
        else:
            # Wenn die vorherige Spalte in df_LY gleich 0 ist, um 1 erhöhen
            df_LO_anwendung.at[index, str(current_month)] = df_LO_anwendung.at[index, str(previous_month)] + 1
            
#Nan durch Nullen ersetzen
df_LO_anwendung = df_LO_anwendung.fillna(0)
df_LO_anwendung = df_LO_anwendung.drop("0", axis=1)

In [36]:
df_LO_anwendung.head()

,iid,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,100056041,1.0,1.0,1.0,1.0,1.0,2.0,3.0,1.0,2.0,1.0,2.0,1.0,2.0,3.0,1.0
1,100057957,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0
2,100059553,1.0,2.0,3.0,1.0,1.0,1.0,1.0,2.0,3.0,1.0,2.0,3.0,1.0,1.0,1.0
3,100062565,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,100062713,1.0,1.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0


## Ausgabe der Daten

In [37]:
path = x
filename = 'Pivot_Trans_Privat_ohneApple_Langzeitkunde.csv'

df_pivoted_t.to_csv(os.path.join(path, filename), index=False)

In [38]:
path = x
filename = 'Pivot_LY_Privat_ohneApple_Langzeitkunde.csv'

df_LY_anwendung.to_csv(os.path.join(path, filename), index=False)

In [39]:
path = x
filename = 'Pivot_HY_Privat_ohneApple_Langzeitkunde.csv'

df_pivoted_HY.to_csv(os.path.join(path, filename), index=False)

In [40]:
path = x
filename = 'Pivot_NO_Privat_ohneApple_Langzeitkunde.csv'

df_pivoted_NO.to_csv(os.path.join(path, filename), index=False)

In [41]:
path = x
filename = 'Pivot_LO_Privat_ohneApple_Langzeitkunde.csv'

df_LO_anwendung.to_csv(os.path.join(path, filename), index=False)